In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
from plotly.subplots import make_subplots

from sqlalchemy import create_engine


In [2]:
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = 'BRtTaqYiJyr29WXN'
TABLE_SCHEMA = 'diploma'
ENGINE = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')

In [3]:
# Посмотрим какие таблицы существуют в нужной схеме
df = pd.read_sql(f'''  
    SELECT *
    FROM information_schema.tables
    WHERE table_schema='{TABLE_SCHEMA}'
''', con=ENGINE)
df

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,game-analytics,diploma,geo_daily_report,BASE TABLE,None,None,None,None,None,YES,NO,None
1,game-analytics,diploma,users,BASE TABLE,None,None,None,None,None,YES,NO,None


## Знакомство с данными

## Создадим запросы для таблиц  по нужным условиям в sql для предварительного ознакомления

In [5]:
cohort_start ='2022-04-01'
cohort_end ='2022-05-01'
project_id = 'MiT'

geo_daily_report_sql = pd.read_sql(f'''
SELECT *
FROM {TABLE_SCHEMA}.geo_daily_report       
WHERE project_id = '{project_id}'
AND event_date >='{cohort_start}'
AND event_date < '{cohort_end}'
''', con=ENGINE)
geo_daily_report_sql.head(2)

,event_date,project_id,platform,country_code,media_source,campaign,impressions,clicks,installs,loyal_users,total_cost,af_ad_revenue_users,af_ad_revenue_events,af_ad_revenue_usd,af_purchase_users,af_purchase_events,af_purchase_usd,af_revenue_users,af_revenue_events,af_revenue_usd
0,2022-04-01,MiT,android,AE,Organic,None,NaN,NaN,2,1,NaN,0,4,0.392066,0,0,0.0,None,None,None
1,2022-04-01,MiT,android,BR,Organic,None,NaN,NaN,8,3,NaN,0,0,0.000000,0,0,0.0,None,None,None


## Создадим для удобства пару функций для знакомства с данными

In [7]:
# Функция выводит инфу о таблице
def df_info(df):
    '''
    Функция работает только  с DataFrame.
    Выводит информацию,    
    проверка на дубликаты,
    проверка на пропуски,
    выводит первые 5 строк,
    выводит последние 5 строк.
    
    '''
    display(df.info())
    display(df.duplicated().sum())
    display(df.isnull().sum())
    display(df.head())
    display(df.tail())

In [8]:
# Функция выводит инфу о поле с категориальными данными
def df_column_categorial(df_name, df_column_name):
    '''
    Функция работает только с DataFrame.
    На вход принимает 2 значения в кавычках: "имя DataFrame", "имя столбца DataFrame".
    Выводит кол-во значений в столбце, 
    выводит кол-во уникальных значений в столбце,
    выводит уникальные значений в столбце
    выводит долю относительных значений в столбце.
    
    '''
    str_cnt = '{0}.{1}.count()'.format(df_name, df_column_name)
    str_nunique = '{0}.{1}.nunique()'.format(df_name, df_column_name)
    str_unique = '{0}.{1}.unique()'.format(df_name, df_column_name)
    str_value_cnt = '{0}.{1}.value_counts(normalize=True).to_frame()'.format(df_name, df_column_name)
    display (eval(str_cnt), eval(str_nunique), eval(str_unique), eval(str_value_cnt))
    

## Подробнее ознакомимся с датафрэймом geo_daily_report_sql

In [103]:
# Инфа о DataFrame
df_info(geo_daily_report_sql)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7572 entries, 0 to 7571
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   event_date            7572 non-null   object 
 1   project_id            7572 non-null   object 
 2   platform              7572 non-null   object 
 3   country_code          7572 non-null   object 
 4   media_source          7572 non-null   object 
 5   campaign              7572 non-null   object 
 6   impressions           1687 non-null   float64
 7   clicks                314 non-null    float64
 8   installs              7572 non-null   int64  
 9   loyal_users           7572 non-null   int64  
 10  total_cost            169 non-null    float64
 11  af_ad_revenue_users   7572 non-null   int64  
 12  af_ad_revenue_events  7572 non-null   int64  
 13  af_ad_revenue_usd     7572 non-null   float64
 14  af_purchase_users     7572 non-null   int64  
 15  af_purchase_events   

None

0

event_date                 0
project_id                 0
platform                   0
country_code               0
media_source               0
campaign                   0
impressions             5885
clicks                  7258
installs                   0
loyal_users                0
total_cost              7403
af_ad_revenue_users        0
af_ad_revenue_events       0
af_ad_revenue_usd          0
af_purchase_users          0
af_purchase_events         0
af_purchase_usd            0
af_revenue_users        7572
af_revenue_events       7572
af_revenue_usd          7572
dtype: int64

,event_date,project_id,platform,country_code,media_source,campaign,impressions,clicks,installs,loyal_users,total_cost,af_ad_revenue_users,af_ad_revenue_events,af_ad_revenue_usd,af_purchase_users,af_purchase_events,af_purchase_usd,af_revenue_users,af_revenue_events,af_revenue_usd
0,2022-04-01,MiT,android,AE,Organic,None,NaN,NaN,2,1,NaN,0,4,0.392066,0,0,0.0,None,None,None
1,2022-04-01,MiT,android,BR,Organic,None,NaN,NaN,8,3,NaN,0,0,0.000000,0,0,0.0,None,None,None
2,2022-04-01,MiT,android,CR,Organic,None,NaN,NaN,1,0,NaN,0,0,0.000000,0,0,0.0,None,None,None
3,2022-04-01,MiT,android,EG,Organic,None,NaN,NaN,5,3,NaN,0,3,0.024070,0,0,0.0,None,None,None
4,2022-04-01,MiT,android,ID,Organic,None,NaN,NaN,11,4,NaN,0,27,2.014312,0,0,0.0,None,None,None


,event_date,project_id,platform,country_code,media_source,campaign,impressions,clicks,installs,loyal_users,total_cost,af_ad_revenue_users,af_ad_revenue_events,af_ad_revenue_usd,af_purchase_users,af_purchase_events,af_purchase_usd,af_revenue_users,af_revenue_events,af_revenue_usd
7567,2022-05-31,MiT,android,PE,Organic,None,NaN,NaN,5,2,NaN,0,0,0.0,0,0,0.0,None,None,None
7568,2022-05-31,MiT,android,PY,Organic,None,NaN,NaN,1,0,NaN,0,0,0.0,0,0,0.0,None,None,None
7569,2022-05-31,MiT,android,SD,Organic,None,NaN,NaN,3,2,NaN,0,0,0.0,0,0,0.0,None,None,None
7570,2022-05-31,MiT,android,TR,Organic,None,NaN,NaN,30,11,NaN,0,0,0.0,0,0,0.0,None,None,None
7571,2022-05-31,MiT,android,UZ,Organic,None,NaN,NaN,1,0,NaN,0,0,0.0,0,0,0.0,None,None,None


## Выведем для наглядности уникальные значения по всем полям датафрэйма geo_daily_report_sql

In [104]:
#Пропишем опцию для максимального вывода
pd.set_option('max_colwidth', 1000)

#Получим уникальные значения по каждому столбцу для ознакомления с датафреймом geo_daily_report_sql в общем виде
pd.DataFrame(geo_daily_report_sql.apply(pd.unique)).rename(columns={0:'Unique values'})

,Unique values
event_date,"[2022-04-01, 2022-04-02, 2022-04-03, 2022-04-04, 2022-04-05, 2022-04-06, 2022-04-07, 2022-04-08, 2022-04-09, 2022-04-10, 2022-04-14, 2022-04-11, 2022-04-12, 2022-04-13, 2022-04-15, 2022-04-16, 2022-04-17, 2022-04-18, 2022-04-19, 2022-04-20, 2022-04-21, 2022-04-22, 2022-04-23, 2022-04-24, 2022-04-25, 2022-04-26, 2022-04-27, 2022-04-28, 2022-04-29, 2022-04-30, 2022-05-01, 2022-05-02, 2022-05-03, 2022-05-04, 2022-05-05, 2022-05-06, 2022-05-07, 2022-05-08, 2022-05-11, 2022-05-09, 2022-05-10, 2022-05-12, 2022-05-13, 2022-05-14, 2022-05-15, 2022-05-16, 2022-05-17, 2022-05-18, 2022-05-19, 2022-05-20, 2022-05-21, 2022-05-22, 2022-05-23, 2022-05-24, 2022-05-25, 2022-05-26, 2022-05-27, 2022-05-28, 2022-05-29, 2022-05-30, 2022-05-31]"
project_id,[MiT]
platform,[android]
country_code,"[AE, BR, CR, EG, ID, IR, LT, NL, PH, SA, TH, UK, ZA, BG, CA, ES, HR, IN, JM, MY, PL, RO, AT, BH, CL, JP, PK, SI, TW, VE, AU, ME, NP, UA, AL, IS, MX, None, RU, ZM, DE, GR, HU, IL, KZ, NG, US, BE, GA, IT, MA, PE, SR, CO, LB, PA, PT, SC, TC, AR, CZ, EE, FR, IE, KM, MM, NO, PR, RS, SG, VI, GF, NZ, SV, TR, UY, DK, ET, JE, LY, BB, BN, CH, IQ, KR, MV, SO, BO, GE, GY, KE, QA, SD, AO, SN, VN, GH, HK, KN, MD, ...]"
media_source,"[Organic, bytedanceglobal_int, googleadwords_int, Facebook Ads]"
campaign,"[None, MiT_EM_AND_SEA_MAI_IT1_2803, MiT_EM_AND_US_Purchase_IT1_2703, MiT_EM_AND_T1_Purchase_IT2_0604, MiT_EM_AND_SEA_Purchase_IT2_0604, MiT_AND_First_Open_FR_GR_1103, MiT_MS_AND_T1_PURCH_AEO_IT1_1504, MiT_MS_AND_T1_PURCH_AEO_IT1_2203, MiT_MS_AND_US_PURCH_AEO_IT1_1504, MiT_EM_AND_T1_Purchase_IT2_0605, MiT_EM_AND_US_MAI_IT3_1705, MiT_EM_AND_US_MAI_IT4_2505, MiT_EM_AND_T1_Purchase_eventcap_2405, MiT_MS_AND_US_PURCH_AAA_IT1_2505, MiT_EM_AND_US_MAI_IT1_2203, MiT_EM_AND_T1_MAI_IT1_2203, MiT_MS_AND_US_PURCH_AEO_IT1_2203, MiT_AND_First_Open_UK_GR_1003, MiT_AND_First_Open_DE_GR_1003, MiT_MS_AND_US_MAI_AAA_IT1_2805, MiT_MS_AND_US_AEO_CBO_IT1_3105]"
impressions,"[nan, 2.0, 3.0, 1.0, 7.0, 8.0, 18.0, 53939.0, 19862.0, 11.0, 1242.0, 12.0, 17442.0, 77.0, 14682.0, 4.0, 13.0, 18445.0, 23.0, 55.0, 226.0, 587.0, 448.0, 1280.0, 28848.0, 445.0, 9.0, 1975.0, 20720.0, 60.0, 61.0, 22796.0, 6.0, 84.0, 87.0, 14975.0, 7023.0, 5293.0, 11205.0, 254.0, 24608.0, 198.0, 15.0, 2179.0, 117.0, 5.0, 20.0, 28.0, 10041.0, 53547.0, 10.0, 6657.0, 21.0, 4774.0, 26.0, 29.0, 30.0, 46.0, 329.0, 1787.0, 18351.0, 18259.0, 430.0, 1403.0, 1887.0, 1388.0, 1940.0, 23154.0, 274.0, 469.0, 1532.0, 18883.0, 49.0, 62.0, 23248.0, 65.0, 20680.0, 20689.0, 9028.0, 38.0, 25673.0, 89.0, 32171.0, 75.0, 16.0, 18965.0, 24762.0, 57.0, 17874.0, 20765.0, 390.0, 465.0, 6143.0, 72.0, 22638.0, 22491.0, 10708.0, 4534.0, 4686.0, 14.0, ...]"
clicks,"[nan, 1.0, 2076.0, 199.0, 25.0, 352.0, 372.0, 328.0, 3.0, 8.0, 9.0, 10.0, 417.0, 7.0, 2.0, 268.0, 4.0, 248.0, 82.0, 49.0, 135.0, 615.0, 6.0, 176.0, 2344.0, 247.0, 246.0, 43.0, 269.0, 19.0, 37.0, 22.0, 28.0, 595.0, 14.0, 36.0, 270.0, 252.0, 154.0, 186.0, 88.0, 173.0, 250.0, 249.0, 414.0, 275.0, 407.0, 98.0, 5.0, 190.0, 114.0, 35.0, 52.0, 554.0, 792.0, 627.0, 117.0, 206.0, 180.0, 405.0, 16.0, 393.0, 543.0, 24.0, 177.0, 179.0, 11.0, 235.0, 18.0, 399.0, 13.0, 271.0, 211.0, 93.0, 187.0, 110.0, 119.0, 539.0, 34.0, 684.0, 29.0, 231.0, 44.0, 81.0, 315.0, 26.0, 259.0, 100.0, 95.0, 202.0, 308.0, 125.0, 78.0, 489.0, 687.0, 12.0, 306.0, 256.0, 300.0, 207.0, ...]"
installs,"[2, 8, 1, 5, 11, 0, 19, 4, 514, 3, 6, 14, 18, 64, 7, 25, 9, 66, 86, 108, 36, 54, 23, 34, 139, 43, 10, 140, 26, 21, 178, 24, 128, 12, 22, 81, 37, 15, 94, 28, 70, 35, 17, 118, 32, 13, 133, 168, 31, 101, 45, 160, 33, 67, 63, 89, 49, 152, 110, 202, 242, 20, 245, 90, 115, 57, 30, 119, 145, 51, 53, 283, 39, 129, 211, 131, 151, 386, 48, 260, 16, 446, 113, 27, 302, 44, 646, 96, 59, 41, 130, 105, 261, 29, 78, 71, 42, 111, 38, 55, ...]"
loyal_users,"[1, 3, 0, 4, 13, 2, 154, 8, 12, 25, 6, 5, 15, 18, 26, 59, 20, 28, 10, 58, 70, 11, 67, 9, 46, 7, 14, 43, 23, 49, 17, 22, 40, 66, 82, 60, 16, 93, 36, 38, 54, 74, 90, 105, 131, 13

#### Предположим, что данный датафрейм это таблица данных траффика со следующими полями:


In [105]:
# Список столбцов
geo_daily_report_sql.columns

Index(['event_date', 'project_id', 'platform', 'country_code', 'media_source',
       'campaign', 'impressions', 'clicks', 'installs', 'loyal_users',
       'total_cost', 'af_ad_revenue_users', 'af_ad_revenue_events',
       'af_ad_revenue_usd', 'af_purchase_users', 'af_purchase_events',
       'af_purchase_usd', 'af_revenue_users', 'af_revenue_events',
       'af_revenue_usd'],
      dtype='object')

In [11]:
test_data = pd.Series({'event_date' :'дата события', 'project_id' :'название проекта',
                       'platform':'платформа', 'country_code' :'код страны',
                       'media_source':'маркетинговые каналы', 'campaign' :'тип события установки',
                       'impressions' :'кол-во показов рекламы', 'clicks' :'кол-во кликов по рекламе',   
                       'installs' :'установки', 'loyal_users' :'лояльные пользователи',
                       'total_cost' :'общие затраты', 'af_ad_revenue_users' :'пользователи рекламы',
                       'af_ad_revenue_events' :'события выручки от рекламы', 'af_ad_revenue_usd' :'выручка от рекламы в usd',  
                       'af_purchase_users' :'платящие пользователи', 'af_purchase_events' :'события платежей',
                       'af_purchase_usd' :'iap платежи в usd', 'af_revenue_users' :'выручка пользователей', 
                       'af_revenue_events' :'события выручки', 'af_revenue_usd' :'выручка в usd',   
                     })
test_data

event_date                            дата события
project_id                        название проекта
platform                                 платформа
country_code                            код страны
media_source                  маркетинговые каналы
campaign                     тип события установки
impressions                 кол-во показов рекламы
clicks                    кол-во кликов по рекламе
installs                                 установки
loyal_users                  лояльные пользователи
total_cost                           общие затраты
af_ad_revenue_users           пользователи рекламы
af_ad_revenue_events    события выручки от рекламы
af_ad_revenue_usd         выручка от рекламы в usd
af_purchase_users            платящие пользователи
af_purchase_events                события платежей
af_purchase_usd                  iap платежи в usd
af_revenue_users             выручка пользователей
af_revenue_events                  события выручки
af_revenue_usd                 


## Исследование столбца "event_date"

In [12]:
# Для предварительного исследования создадим Series и приведем типы данных к формату "Y-M-D" 
event_date_series  = pd.to_datetime(geo_daily_report_sql['event_date'], format='%Y-%m-%d')

In [13]:
# min и max дата
display(event_date_series.min(), event_date_series.max())

Timestamp('2022-04-01 00:00:00')

Timestamp('2022-04-30 00:00:00')

***В датасете присутствуют данные c 2022-04-01 по 2022-05-31, но нужны только за 2022-04 месяц.***

## Исследование столбца 'project_id'
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят

In [14]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql','project_id')

3281

1

array(['MiT'], dtype=object)

,project_id
MiT,1.0


***Представлена платформа только проект MiT***

## Исследование столбца 'platform'
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят


In [15]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql','platform')

3281

1

array(['android'], dtype=object)

,platform
android,1.0


***Представлена платформа только андроид***

## Исследование столбца 'country_code '
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят
подробнее можно ознакомиться: https://countrycode.org/

In [40]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql','country_code')

3281

188

array(['AE', 'BR', 'CR', 'EG', 'ID', 'IR', 'LT', 'NL', 'PH', 'SA', 'TH',
       'UK', 'ZA', 'BG', 'CA', 'ES', 'HR', 'IN', 'JM', 'MY', 'PL', 'RO',
       'AT', 'BH', 'CL', 'JP', 'PK', 'SI', 'TW', 'VE', 'AU', 'ME', 'NP',
       'UA', 'AL', 'IS', 'MX', 'None', 'RU', 'ZM', 'DE', 'GR', 'HU', 'IL',
       'KZ', 'NG', 'US', 'BE', 'GA', 'IT', 'MA', 'PE', 'SR', 'CO', 'LB',
       'PA', 'PT', 'SC', 'TC', 'AR', 'CZ', 'EE', 'FR', 'IE', 'KM', 'MM',
       'NO', 'PR', 'RS', 'SG', 'VI', 'GF', 'NZ', 'SV', 'TR', 'UY', 'DK',
       'ET', 'JE', 'LY', 'BB', 'BN', 'CH', 'IQ', 'KR', 'MV', 'SO', 'BO',
       'GE', 'GY', 'KE', 'QA', 'SD', 'AO', 'SN', 'VN', 'GH', 'HK', 'KN',
       'MD', 'ZW', 'CN', 'DO', 'KW', 'SE', 'SX', 'MF', 'PS', 'SK', 'TT',
       'CY', 'BS', 'FJ', 'DZ', 'HT', 'YE', '', 'SS', 'BD', 'JO', 'KY',
       'BY', 'KG', 'LU', 'RE', 'UZ', 'TZ', 'CU', 'MK', 'VC', 'BW', 'LK',
       'MU', 'BJ', 'BA', 'GU', 'MQ', 'TN', 'GD', 'GG', 'IM', 'GT', 'AZ',
       'FI', 'OM', 'AM', 'AF', 'MN', 'EC', 'AS', 'B

,country_code
US,0.034441
UK,0.024992
DE,0.023468
CA,0.020421
FR,0.019506
...,...
AI,0.000305
BI,0.000305
AS,0.000305
DM,0.000305


In [42]:
# Построим график в plotly 
df = geo_daily_report_sql.country_code.value_counts(normalize=True).reset_index()
fig = px.bar(df.loc[:10],
             x='index',
             y='country_code',
             title='Топ 10 стран, 30% приходиться на лидера US')
fig.update_layout(width=600, 
                  height=400)
fig.show()

***В датасете представлены 188 стран, из 30% приходиться на лидера US***


## Исследование столбца 'media_source'
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят

In [117]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql','media_source')

7572

4

array(['Organic', 'bytedanceglobal_int', 'googleadwords_int',
       'Facebook Ads'], dtype=object)

,media_source
Organic,0.771130
bytedanceglobal_int,0.220549
Facebook Ads,0.007660
googleadwords_int,0.000660


In [18]:
# Построим график в plotly 


# С учетом органики
df1 = geo_daily_report_sql.media_source.value_counts(normalize=True).reset_index()

# Без учета органики
df2 = geo_daily_report_sql.query('media_source != "Organic"').media_source.value_counts(normalize=True).reset_index()

fig1 = px.bar(df1.loc[:5],             
             x='index', 
             y='media_source'
             )

fig2 = px.bar(df2.loc[:5],             
             x='index', 
             y='media_source'
             )

fig = make_subplots(rows=1, 
                    cols=2, 
                    
                    subplot_titles=('22% в данных лидирует канал <br> MiT_EM_AND_US_Purchase_IT1_2703 <br> после органики', 
                                    '96% в данных лидирует канал <br> MiT_EM_AND_US_Purchase_IT1_2703 <br> исключая органику')
                   )

fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)

fig.update_layout(width=900, 
                  height=400, 
                  margin=dict(l=20, r=20, t=100, b=0),
)

fig.show()

***В датасете представлены 4 маркетинговых канала включая органику, лидирует bytedanceglobal_int = 22% после органики, а 96% лидирует bytedanceglobal_int исключая органику***

## Исследование столбца 'campaign'
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят

In [172]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql_campaign_str','campaign')

7572

21

array(['None', 'MiT_EM_AND_SEA_MAI_IT1_2803',
       'MiT_EM_AND_US_Purchase_IT1_2703',
       'MiT_EM_AND_T1_Purchase_IT2_0604',
       'MiT_EM_AND_SEA_Purchase_IT2_0604',
       'MiT_AND_First_Open_FR_GR_1103',
       'MiT_MS_AND_T1_PURCH_AEO_IT1_1504',
       'MiT_MS_AND_T1_PURCH_AEO_IT1_2203',
       'MiT_MS_AND_US_PURCH_AEO_IT1_1504',
       'MiT_EM_AND_T1_Purchase_IT2_0605', 'MiT_EM_AND_US_MAI_IT3_1705',
       'MiT_EM_AND_US_MAI_IT4_2505',
       'MiT_EM_AND_T1_Purchase_eventcap_2405',
       'MiT_MS_AND_US_PURCH_AAA_IT1_2505', 'MiT_EM_AND_US_MAI_IT1_2203',
       'MiT_EM_AND_T1_MAI_IT1_2203', 'MiT_MS_AND_US_PURCH_AEO_IT1_2203',
       'MiT_AND_First_Open_UK_GR_1003', 'MiT_AND_First_Open_DE_GR_1003',
       'MiT_MS_AND_US_MAI_AAA_IT1_2805', 'MiT_MS_AND_US_AEO_CBO_IT1_3105'],
      dtype=object)

,campaign
None,0.771395
MiT_EM_AND_US_Purchase_IT1_2703,0.085975
MiT_EM_AND_T1_Purchase_IT2_0604,0.045034
MiT_EM_AND_US_MAI_IT3_1705,0.027866
MiT_EM_AND_US_MAI_IT4_2505,0.016508
MiT_EM_AND_SEA_Purchase_IT2_0604,0.014659
MiT_EM_AND_T1_Purchase_IT2_0605,0.011622
MiT_EM_AND_SEA_MAI_IT1_2803,0.010829
MiT_EM_AND_T1_Purchase_eventcap_2405,0.006471
MiT_MS_AND_T1_PURCH_AEO_IT1_1504,0.002377


## Гипотеза 2 
# Убедимся, что записи campaign =="None" являются органикой

In [22]:
# Убедимся, что записи campaign =="" являются органикой
geo_daily_report_sql.query('campaign =="None"')[['media_source','campaign']].value_counts(normalize=True)


media_source  campaign
Organic       None        0.999564
Facebook Ads  None        0.000436
dtype: float64

In [23]:
# Проверим пропуски campaign по каналу Facebook Ads
geo_daily_report_sql.query('(campaign =="None") & (media_source == "Facebook Ads")')

,event_date,project_id,platform,country_code,media_source,campaign,impressions,clicks,installs,loyal_users,total_cost,af_ad_revenue_users,af_ad_revenue_events,af_ad_revenue_usd,af_purchase_users,af_purchase_events,af_purchase_usd,af_revenue_users,af_revenue_events,af_revenue_usd
627,2022-04-16,MiT,android,DE,Facebook Ads,None,NaN,NaN,1,0,NaN,0,0,0.0,0,0,0.0,None,None,None


In [24]:
# Проверим пропуски campaign по каналу Facebook Ads
geo_daily_report_sql.query('(campaign =="None") & (media_source == "bytedanceglobal_int")')

,event_date,project_id,platform,country_code,media_source,campaign,impressions,clicks,installs,loyal_users,total_cost,af_ad_revenue_users,af_ad_revenue_events,af_ad_revenue_usd,af_purchase_users,af_purchase_events,af_purchase_usd,af_revenue_users,af_revenue_events,af_revenue_usd


In [25]:
# Построим график в plotly 


# С учетом органики
df1 = geo_daily_report_sql.campaign.value_counts(normalize=True).reset_index()

# Без учета органики
df2 = geo_daily_report_sql.query('campaign != "None"').campaign.value_counts(normalize=True).reset_index()

fig1 = px.bar(df1.loc[:5],             
             x='index', 
             y='campaign'
             )

fig2 = px.bar(df2.loc[:5],             
             x='index', 
             y='campaign'
             )

fig = make_subplots(rows=1, 
                    cols=2, 
                    
                    subplot_titles=('8% в данных лидирует канал <br> MiT_EM_AND_US_Purchase_IT1_2703 <br> после органики', 
                                    '38% в данных лидирует канал <br> MiT_EM_AND_US_Purchase_IT1_2703 <br> исключая органику')
                   )

fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)

fig.update_layout(width=900, 
                  height=400, 
                  margin=dict(l=20, r=20, t=100, b=0),
)

fig.show()

***В датасете представлены 21 рекламных кампаний включая органику, лидирует MiT_EM_AND_US_Purchase_IT1_2703 = 5% после органики, а 96% лидирует bytedanceglobal_int исключая органику. Так же есть 0,01% -неполные данные "None" по кампании "Facebook Ads" и 0,01% -неполные данные "None" по кампании "bytedanceglobal_int". Добавим это в баг_репорт.***

### Баг_репорт_№4:
В таблице 'geo_daily_report_sql':
- 0,01% -неполные данные campaign =="None" по кампании "Facebook Ads",
- 0,01% -неполные данные campaign =="None" по кампании "bytedanceglobal_int".

## Исследование столбца 'impressions'
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят

In [28]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql','impressions')

965

155

array([       nan, 2.0000e+00, 3.0000e+00, 1.0000e+00, 7.0000e+00,
       8.0000e+00, 1.8000e+01, 5.3939e+04, 1.9862e+04, 1.1000e+01,
       1.2420e+03, 1.2000e+01, 1.7442e+04, 7.7000e+01, 1.4682e+04,
       4.0000e+00, 1.3000e+01, 1.8445e+04, 2.3000e+01, 5.5000e+01,
       2.2600e+02, 5.8700e+02, 4.4800e+02, 1.2800e+03, 2.8848e+04,
       4.4500e+02, 9.0000e+00, 1.9750e+03, 2.0720e+04, 6.0000e+01,
       6.1000e+01, 2.2796e+04, 1.5000e+01, 5.3547e+04, 1.0000e+01,
       6.0000e+00, 5.0000e+00, 6.6570e+03, 2.1000e+01, 4.7740e+03,
       2.6000e+01, 2.9000e+01, 3.0000e+01, 4.6000e+01, 3.2900e+02,
       1.7870e+03, 1.8351e+04, 1.8259e+04, 4.3000e+02, 1.4030e+03,
       1.8870e+03, 1.3880e+03, 1.9400e+03, 2.3154e+04, 2.7400e+02,
       4.6900e+02, 1.5320e+03, 1.8883e+04, 4.9000e+01, 6.2000e+01,
       2.3248e+04, 1.6000e+01, 1.8965e+04, 2.4762e+04, 5.7000e+01,
       1.7874e+04, 2.0765e+04, 3.9000e+02, 4.6500e+02, 6.1430e+03,
       7.2000e+01, 1.5398e+04, 4.9280e+03, 6.3000e+01, 2.1047e

,impressions
1.0,0.396891
2.0,0.200000
3.0,0.078756
4.0,0.044560
5.0,0.031088
...,...
18965.0,0.001036
19862.0,0.001036
23248.0,0.001036
62.0,0.001036


## Гипотзеа: None в поле impressions это органика

In [26]:
# Посмотрим, что за записи с пропусками в поле impressions
geo_daily_report_sql[geo_daily_report_sql.impressions.isnull()].head(3)

,event_date,project_id,platform,country_code,media_source,campaign,impressions,clicks,installs,loyal_users,total_cost,af_ad_revenue_users,af_ad_revenue_events,af_ad_revenue_usd,af_purchase_users,af_purchase_events,af_purchase_usd,af_revenue_users,af_revenue_events,af_revenue_usd
0,2022-04-01,MiT,android,AE,Organic,None,NaN,NaN,2,1,NaN,0,4,0.392066,0,0,0.0,None,None,None
1,2022-04-01,MiT,android,BR,Organic,None,NaN,NaN,8,3,NaN,0,0,0.000000,0,0,0.0,None,None,None
2,2022-04-01,MiT,android,CR,Organic,None,NaN,NaN,1,0,NaN,0,0,0.000000,0,0,0.0,None,None,None


***Видим, что это записи органики***

In [30]:
# Построим график в plotly 


# С учетом органики
df1 = geo_daily_report_sql.impressions.value_counts(normalize=True).reset_index()

# Без учета органики
df2 = geo_daily_report_sql.query('campaign != "None"').impressions.value_counts(normalize=True).reset_index()

fig1 = px.bar(df1.loc[:5],             
             x='index', 
             y='impressions'
             )

fig2 = px.bar(df2.loc[:5],             
             x='index', 
             y='impressions'
             )

fig = make_subplots(rows=1, 
                    cols=2, 
                    
                    subplot_titles=('9% в данных лидирует показ <br> рекламы 1 раз после органики', 
                                    '41% в данных лидирует показ <br> рекламы 1 раз исключая органику')
                   )

fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)

fig.update_layout(width=900, 
                  height=400, 
                  margin=dict(l=20, r=20, t=100, b=0),
)

fig.show()

***В датасете представлены 7572 показов рекламы включая органику, 9% в данных лидирует показ рекламы 1 раз после органики, 41% в данных лидирует показ рекламы 1 раз исключая органику.***

***В датасете 78% всех записей в поле impressions представляет органика, причем только 9% из всех данных составлет показ рекламы 1 раз.***

## Исследование столбца 'clicks'
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят

In [298]:
# Приведем impressions в str вместо float для отображения nan в графике
geo_daily_report_sql_clicks_str = geo_daily_report_sql.copy()
geo_daily_report_sql_clicks_str['clicks'] = geo_daily_report_sql_clicks_str['clicks'].astype(str)


In [32]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql','clicks')

183

86

array([      nan, 1.000e+00, 2.076e+03, 1.990e+02, 2.500e+01, 3.520e+02,
       3.720e+02, 3.280e+02, 3.000e+00, 8.000e+00, 9.000e+00, 1.000e+01,
       4.170e+02, 7.000e+00, 2.000e+00, 2.680e+02, 2.344e+03, 4.000e+00,
       2.470e+02, 2.460e+02, 6.000e+00, 4.300e+01, 2.480e+02, 2.690e+02,
       1.900e+01, 3.700e+01, 2.200e+01, 2.800e+01, 5.950e+02, 1.400e+01,
       3.600e+01, 2.700e+02, 2.520e+02, 2.490e+02, 4.140e+02, 2.750e+02,
       4.070e+02, 9.800e+01, 5.000e+00, 1.800e+02, 1.170e+02, 4.050e+02,
       1.600e+01, 3.930e+02, 5.430e+02, 2.400e+01, 1.770e+02, 1.790e+02,
       1.100e+01, 2.350e+02, 1.800e+01, 3.990e+02, 1.300e+01, 2.710e+02,
       2.310e+02, 4.400e+01, 8.100e+01, 3.150e+02, 2.600e+01, 2.590e+02,
       1.000e+02, 9.500e+01, 2.020e+02, 1.200e+01, 3.060e+02, 2.560e+02,
       3.000e+02, 2.070e+02, 3.400e+01, 3.900e+01, 3.380e+02, 3.610e+02,
       2.190e+02, 4.830e+02, 2.090e+02, 1.500e+01, 3.100e+01, 3.500e+01,
       2.640e+02, 2.676e+03, 2.570e+02, 3.000e+01, 

,clicks
1.0,0.311475
2.0,0.049180
4.0,0.032787
25.0,0.021858
3.0,0.021858
...,...
393.0,0.005464
2076.0,0.005464
177.0,0.005464
179.0,0.005464


## Гипотзеа: NA в поле clicks это органика

In [33]:
# Посмотрим, что за записи с пропусками в поле impressions
geo_daily_report_sql[geo_daily_report_sql.clicks.isnull()].head(3)

,event_date,project_id,platform,country_code,media_source,campaign,impressions,clicks,installs,loyal_users,total_cost,af_ad_revenue_users,af_ad_revenue_events,af_ad_revenue_usd,af_purchase_users,af_purchase_events,af_purchase_usd,af_revenue_users,af_revenue_events,af_revenue_usd
0,2022-04-01,MiT,android,AE,Organic,None,NaN,NaN,2,1,NaN,0,4,0.392066,0,0,0.0,None,None,None
1,2022-04-01,MiT,android,BR,Organic,None,NaN,NaN,8,3,NaN,0,0,0.000000,0,0,0.0,None,None,None
2,2022-04-01,MiT,android,CR,Organic,None,NaN,NaN,1,0,NaN,0,0,0.000000,0,0,0.0,None,None,None


***Видим, что это записи органики***

# Исследование столбца 'installs'
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят

In [39]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql','installs')

3281

112

array([  2,   8,   1,   5,  11,   0,  19,   4, 514,   3,   6,  14,  18,
        64,   7,  25,   9,  66,  86, 108,  36,  54,  23,  34, 139,  43,
        10, 140,  26,  21, 178,  24, 128,  12,  22,  81,  37,  15,  94,
        28,  70,  35,  17, 118,  32,  13, 133, 646,  48,  31,  16,  27,
       101,  44,  49,  89,  96, 115,  59,  41, 130, 105,  55,  42,  76,
       158,  20, 124,  53,  67, 156,  39, 131,  52, 142,  88,  46,  69,
       114,  45,  63,  58,  73,  77, 154,  92,  38,  95,  47,  57,  61,
        30, 102,  75, 119,  29, 150,  40,  98,  90, 112, 121, 120,  51,
       676, 100, 155, 151, 166, 160,  72,  33], dtype=int64)

,installs
1,0.303871
0,0.272173
2,0.113685
3,0.064919
4,0.041451
...,...
646,0.000305
63,0.000305
133,0.000305
73,0.000305


In [46]:
geo_daily_report_sql.installs.value_counts(normalize=True)

1      0.303871
0      0.272173
2      0.113685
3      0.064919
4      0.041451
         ...   
646    0.000305
63     0.000305
133    0.000305
73     0.000305
33     0.000305
Name: installs, Length: 112, dtype: float64

In [47]:
# Построим график в plotly 
df = geo_daily_report_sql.installs.value_counts(normalize=True).reset_index()
fig = px.bar(df.loc[:10],
             x='index',
             y='installs',
             title='30% записей имеют 1 установку')
fig.update_layout(width=600, 
                  height=400)
fig.show()

***В датасете представлены 3281 записей с инсталлами, 30% записей имеют 1 установку.Так же поле installs коррелирует с инсталлами из таблицы .users***

# Исследование столбца 'loyal_users'
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят

In [48]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql','loyal_users')

3281

69

array([  1,   3,   0,   4,  13,   2, 154,   8,  12,  25,   6,   5,  15,
        18,  26,  59,  20,  28,  10,  58,  70,  11,  67,   9,  46,   7,
        14,  43,  23,  49,  17,  22,  40,  66, 204,  31,  30,  53,  33,
        54,  38,  24,  65,  27,  39,  35,  57,  21,  61,  75,  19,  74,
        76,  42,  41,  34,  16,  92,  55,  45,  37,  29,  62,  56,  48,
       255,  80,  78,  52], dtype=int64)

,loyal_users
0,0.484913
1,0.252362
2,0.078939
3,0.043889
4,0.027126
...,...
70,0.000305
58,0.000305
59,0.000305
154,0.000305


In [51]:
# Построим график в plotly 
df = geo_daily_report_sql.loyal_users.value_counts(normalize=True).reset_index()
fig = px.bar(df.loc[:10],
             x='index',
             y='loyal_users',
             title='48% записей имеют 1 установку')
fig.update_layout(width=600, 
                  height=400)
fig.show()

***В датасете представлены 3281 записей с инсталлами, 48% записей имеют 1 лояльного игрока.***

# Исследование столбца 'total_cost'
В этом столбце непрерывные данные, => статестические инструменты для анализа подойдут

In [52]:
geo_daily_report_sql.total_cost.describe()

count    109.000000
mean      58.414404
std       52.903161
min        0.020000
25%       11.690000
50%       39.330000
75%       98.080000
max      145.000000
Name: total_cost, dtype: float64

In [53]:
# Построим гистограмму в plotly 
fig = px.histogram(geo_daily_report_sql, 
                   x="total_cost", 
                   title='В датасете представлены данные, где медиана <br> total_cost равна 39.33',
                   nbins = 40)
fig.update_layout(width=600, 
                  height=400 ),
fig.show()

In [ ]:
# кол-во платежей 'костов    109.000000'
# большинство транзакций '50%        39.330000'

***В датасете представлены данные, где где медиана total_cost равна 39.33***

# Исследование столбца 'af_ad_revenue_users'
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят

In [54]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql','af_ad_revenue_users')

3281

1

array([0], dtype=int64)

,af_ad_revenue_users
0,1.0


***В датасете представлены данные, где где af_ad_revenue_users 1 запись со значением 0***

# Исследование столбца 'af_ad_revenue_events'
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят

In [55]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql','af_ad_revenue_events')

3281

113

array([  4,   0,   3,  27,   1,  45,   6,   2, 122,   5,  10,   8,   9,
        40,  13,  11,  22,  24,  34,  12,  31,  38, 189,  93,  17,  29,
        52,  30,   7,  36,  60,  69,  15, 133,  37, 131,  14,  19,  54,
        48,  21,  25,  16,  43,  42, 134, 113,  28,  18,  20, 146,  57,
       195,  66,  51,  32,  87,  49,  88,  26,  33,  65,  85,  47, 130,
        72,  71,  46,  73,  58,  50,  81, 183,  70, 123, 180, 179,  39,
        41, 145,  86,  84, 103,  35,  80,  23, 157, 175,  68,  44,  74,
       205,  56,  78,  79,  83,  97,  94, 139, 102, 176, 213,  61, 111,
       121, 299, 256,  64, 247, 151,  90,  75,  96], dtype=int64)

,af_ad_revenue_events
0,0.597988
1,0.091436
2,0.044194
3,0.028040
4,0.018897
...,...
39,0.000305
145,0.000305
86,0.000305
84,0.000305


# Исследование столбца 'af_ad_revenue_usd'
В этом столбце непрерывные данные, => статестические инструменты для анализа подойдут

In [56]:
geo_daily_report_sql.af_ad_revenue_usd.describe()

count    3281.000000
mean        0.415297
std         2.314452
min         0.000000
25%         0.000000
50%         0.000000
75%         0.038977
max        48.476669
Name: af_ad_revenue_usd, dtype: float64

***В датасете представлены 3281 строка с данными, где среднее af_ad_revenue_usd равно 0.415297***

# Исследование столбца 'af_purchase_users'
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят

In [57]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql','af_purchase_users')

3281

11

array([ 0,  3,  1,  8,  2,  6,  5,  4, 10,  7, 12], dtype=int64)

,af_purchase_users
0,0.921061
1,0.046937
2,0.012191
3,0.005181
5,0.003962
4,0.003353
6,0.003048
7,0.002743
8,0.000914
10,0.000305


***В датасете представлены 90% данных, где af_ad_revenue_usd равно 0***

# Исследование столбца 'af_purchase_events'
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят

In [58]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql','af_purchase_events')

3281

22

array([ 0,  7,  3, 15,  2,  4, 11,  6,  8,  1, 10, 16, 21,  5, 23, 12, 13,
        9, 18, 17, 14, 22], dtype=int64)

,af_purchase_events
0,0.913136
1,0.024383
2,0.019201
3,0.008839
4,0.007315
8,0.003657
7,0.003657
5,0.003657
6,0.003353
9,0.002438


In [ ]:
***В датасете представлены 91% данных, где af_purchase_events равно 0***

# Исследование столбца 'af_purchase_usd'
В этом столбце непрерывные данные, => статестические инструменты для анализа подойдут

In [59]:
geo_daily_report_sql.af_purchase_usd.describe()

count    3281.000000
mean        2.118798
std        10.321455
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       139.748590
Name: af_purchase_usd, dtype: float64

***В датасете представлены 3281 строка с данными, где среднее af_purchase_usd равно 2.118798***

# Исследование столбца 'af_revenue_users'
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят

In [60]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql','af_revenue_users')

0

0

array([None], dtype=object)

,af_revenue_users


***В датасете представлены данные, где где af_ad_revenue_users значения None***

# Исследование столбца 'af_revenue_events'
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят

In [61]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql','af_revenue_events')

0

0

array([None], dtype=object)

,af_revenue_events


***В датасете представлены данные, где где af_revenue_events значения None***

# Исследование столбца 'af_revenue_usd'
этот столбец с категориальными данными => статестические инструменты для анализа этого столбца не подходят

In [62]:
# Применим для этих данных ранее созданную функцию
df_column_categorial('geo_daily_report_sql','af_revenue_usd')

0

0

array([None], dtype=object)

,af_revenue_usd


***В датасете представлены данные, где где af_revenue_usd значения None***